## pyspark 그 외 자주사용하는 함수

In [1]:
from pyspark.sql import (
    Row,
    SparkSession)
import pyspark.sql.functions as F

In [2]:
spark=(
    SparkSession
    .builder
    .appName("allinone_study")
    .master("spark://spark-master:7077")
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/29 14:58:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df=spark.read.csv(
    "file:///workspace/data/user_activity.csv",
    header=True,
    inferSchema=True
)
df.show()
df.printSchema()

+------+----+---+-----+-------+-------+-------------+-------------+---------+--------+--------------------+------+-----+
|emp_id|name|age| dept|   city|country|       phone1|       phone2|join_date|end_date|              skills|salary|bonus|
+------+----+---+-----+-------+-------+-------------+-------------+---------+--------+--------------------+------+-----+
|     1| Kim| 28|   IT|  Seoul|   NULL|010-1234-5678|         NULL| 20200115|    NULL|    Python,SQL,Spark|  5000|  500|
|     2| Lee| 35|   IT|  Busan|  Korea|         NULL|010-9876-1111| 20180501|    NULL|   Java,Spring,MySQL|  7000| NULL|
|     3|Park| 17|   HR|  Seoul|  Korea|010-2222-3333|         NULL| 20240301|    NULL|          Excel,HRIS|  3000|  100|
|     4|Choi| 42|Sales|Incheon|  Korea|010-4444-5555|010-9999-8888| 20121010|20231231|Salesforce,CRM,Ne...|  9000| 1200|
|     5|Jung| 67|   HR|  Busan|   NULL|010-7777-6666|         NULL| 20050101|    NULL|Recruiting,Interview|  6000| NULL|
|     6| Han| 31|   IT|  Seoul| 

### lit 
- 상수값을 채운다. 모든 행에 독같은 값을 넣을때사용

- 이때 withColum(새로만들이름, 계산로직객체) 컬럼 추가 수정 
- 주의! 기존 컬럼명과 똑같이 쓰면 덮어쓰기가 된다!!

- 빈컬럼을 만들때 F.lit(None)으로 만들수 있다

### coalesce(col1,col2,...)

- 여러 컬럼중 Null이 아닌 첫번째 값을 가져온다
- Null을 때울때 필수 

In [4]:
# country없으면 Korea로 채우기 + null컬럼 만들기

In [5]:
df1=df.withColumn(
    "country_filled",
    F.coalesce(F.col("country"),F.lit("Korea"))
).withColumn(
    "dummy_null",
    F.lit(None)
)
df1.show()

+------+----+---+-----+-------+-------+-------------+-------------+---------+--------+--------------------+------+-----+--------------+----------+
|emp_id|name|age| dept|   city|country|       phone1|       phone2|join_date|end_date|              skills|salary|bonus|country_filled|dummy_null|
+------+----+---+-----+-------+-------+-------------+-------------+---------+--------+--------------------+------+-----+--------------+----------+
|     1| Kim| 28|   IT|  Seoul|   NULL|010-1234-5678|         NULL| 20200115|    NULL|    Python,SQL,Spark|  5000|  500|         Korea|      NULL|
|     2| Lee| 35|   IT|  Busan|  Korea|         NULL|010-9876-1111| 20180501|    NULL|   Java,Spring,MySQL|  7000| NULL|         Korea|      NULL|
|     3|Park| 17|   HR|  Seoul|  Korea|010-2222-3333|         NULL| 20240301|    NULL|          Excel,HRIS|  3000|  100|         Korea|      NULL|
|     4|Choi| 42|Sales|Incheon|  Korea|010-4444-5555|010-9999-8888| 20121010|20231231|Salesforce,CRM,Ne...|  9000| 120

### when otherwise

- when(조건,참일때값).otherwise(거짓일때값)

In [6]:
df2=df1.withColumn(
    "emp_type",
    F.when(F.col("age")<20,"미성년")
    .when(F.col("age")<65,"성인")
    .otherwise("정년")
)
df2.select("name","age","emp_type").show()

+----+---+--------+
|name|age|emp_type|
+----+---+--------+
| Kim| 28|    성인|
| Lee| 35|    성인|
|Park| 17|  미성년|
|Choi| 42|    성인|
|Jung| 67|    정년|
| Han| 31|    성인|
| Seo| 24|    성인|
|Yoon| 55|    성인|
+----+---+--------+



### 문자열 처리

1. substring(col,pos,len) -> 글자 자르기 , 인덱스가 0이 아니라 1!
2. split(col,pattern) -> 문자를 쪼개서 리스트로 만든다.
3. concat(col1,col2) -> 여러문자를 합친다
4. regexp_replace(col,pattern,replacement) -> 정규표현식으로 문자를 찾아 바꾼다

In [7]:
# 입사연도 추출
df.select("name","join_date",F.substring("join_date",1,4).alias("join_year")).show()

+----+---------+---------+
|name|join_date|join_year|
+----+---------+---------+
| Kim| 20200115|     2020|
| Lee| 20180501|     2018|
|Park| 20240301|     2024|
|Choi| 20121010|     2012|
|Jung| 20050101|     2005|
| Han| 20220601|     2022|
| Seo| 20230303|     2023|
|Yoon| 20100101|     2010|
+----+---------+---------+



In [8]:
# 전화번호 정리 
df.select("name","phone1",F.regexp_replace("phone1","-","").alias("clean_phone")).show()

+----+-------------+-----------+
|name|       phone1|clean_phone|
+----+-------------+-----------+
| Kim|010-1234-5678|01012345678|
| Lee|         NULL|       NULL|
|Park|010-2222-3333|01022223333|
|Choi|010-4444-5555|01044445555|
|Jung|010-7777-6666|01077776666|
| Han|         NULL|       NULL|
| Seo|010-1111-2222|01011112222|
|Yoon|         NULL|       NULL|
+----+-------------+-----------+



In [9]:
# 도시,국가 합치기
df1.select(
    F.concat(F.col("city"),F.lit("_"),F.col("country_filled")).alias("location")
).show()

+-------------+
|     location|
+-------------+
|  Seoul_Korea|
|  Busan_Korea|
|  Seoul_Korea|
|Incheon_Korea|
|  Busan_Korea|
|  Seoul_Korea|
|  Seoul_Korea|
|Incheon_Korea|
+-------------+



### 배열 구조

1. explode : split로 쪼갠뒤, 리스트이 요소를 각각의 행으로 만들때 쓴다.
2. size : 배열의크기(길이)반환 , 문자열일때 안되고! 배열일때 가능

In [16]:
# truncate=False
df3=df.withColumn("skill_array",F.split(F.col("skills"),","))
# df3.show()
df3_b=df3.withColumn("size_skills",F.size(F.col("skill_array")))
df3_b.select("skills","skill_array","size_skills").show(truncate=False)

+----------------------------+--------------------------------+-----------+
|skills                      |skill_array                     |size_skills|
+----------------------------+--------------------------------+-----------+
|Python,SQL,Spark            |[Python, SQL, Spark]            |3          |
|Java,Spring,MySQL           |[Java, Spring, MySQL]           |3          |
|Excel,HRIS                  |[Excel, HRIS]                   |2          |
|Salesforce,CRM,Negotiation  |[Salesforce, CRM, Negotiation]  |3          |
|Recruiting,Interview        |[Recruiting, Interview]         |2          |
|Python,Spark,DataEngineering|[Python, Spark, DataEngineering]|3          |
|Marketing,Ads,Content       |[Marketing, Ads, Content]       |3          |
|Python,ML,DeepLearning      |[Python, ML, DeepLearning]      |3          |
+----------------------------+--------------------------------+-----------+



In [18]:
# 별도의 행을 생성해서 Kim이 따로 노는 것을 볼수 있다 
df3_b.select("name",F.explode("skill_array").alias("skill")).show()

+----+---------------+
|name|          skill|
+----+---------------+
| Kim|         Python|
| Kim|            SQL|
| Kim|          Spark|
| Lee|           Java|
| Lee|         Spring|
| Lee|          MySQL|
|Park|          Excel|
|Park|           HRIS|
|Choi|     Salesforce|
|Choi|            CRM|
|Choi|    Negotiation|
|Jung|     Recruiting|
|Jung|      Interview|
| Han|         Python|
| Han|          Spark|
| Han|DataEngineering|
| Seo|      Marketing|
| Seo|            Ads|
| Seo|        Content|
|Yoon|         Python|
+----+---------------+
only showing top 20 rows



## 날짜 처리 

1. current_date 오늘 날짜
2. to_date 문자열을 진짜 날짜 타입으로 변경
3. date_add, date_sub 날짜 더하기 , 빼기
4. datediff(end,start) 두날짜 사이의 차이구하기

In [20]:
df4=df.withColumn(
    "join_dt",
    F.to_date("join_date","yyyyMMdd")
)
df4.select("name","join_date","join_dt").show()
df4.printSchema()

+----+---------+----------+
|name|join_date|   join_dt|
+----+---------+----------+
| Kim| 20200115|2020-01-15|
| Lee| 20180501|2018-05-01|
|Park| 20240301|2024-03-01|
|Choi| 20121010|2012-10-10|
|Jung| 20050101|2005-01-01|
| Han| 20220601|2022-06-01|
| Seo| 20230303|2023-03-03|
|Yoon| 20100101|2010-01-01|
+----+---------+----------+

root
 |-- emp_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- dept: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- phone1: string (nullable = true)
 |-- phone2: string (nullable = true)
 |-- join_date: integer (nullable = true)
 |-- end_date: integer (nullable = true)
 |-- skills: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- bonus: integer (nullable = true)
 |-- join_dt: date (nullable = true)



In [21]:
# Calculate number of working days
df4.select(
    "name",
    F.datediff(F.current_date(),F.col("join_dt")).alias("work_days")
).show()

+----+---------+
|name|work_days|
+----+---------+
| Kim|     2206|
| Lee|     2830|
|Park|      699|
|Choi|     4859|
|Jung|     7698|
| Han|     1338|
| Seo|     1063|
|Yoon|     5872|
+----+---------+



## 숫자

1. round(col,소수점 자릿수) 반올림
2. rand() : 0~1사이 랜덤 실수
3. countDistinct(col) 중복제거 , 개수를 센다. 

In [22]:
df.select(
    "name",
    F.round(
        F.col("salary")+F.coalesce(F.col("bonus"),F.lit(0)),0
    ).alias("total_pay")
).show()

+----+---------+
|name|total_pay|
+----+---------+
| Kim|     5500|
| Lee|     7000|
|Park|     3100|
|Choi|    10200|
|Jung|     6000|
| Han|     7200|
| Seo|     4000|
|Yoon|     9500|
+----+---------+



In [23]:
df.groupBy("dept").agg(
    F.count("*").alias("emp_count"),
    F.avg("salary").alias("avg_salary"),
    F.max("salary").alias("max_salary"),
    F.countDistinct("city").alias("city_count")
).show()

+-----+---------+----------+----------+----------+
| dept|emp_count|avg_salary|max_salary|city_count|
+-----+---------+----------+----------+----------+
|Sales|        2|    6500.0|      9000|         2|
|   HR|        2|    4500.0|      6000|         2|
|   IT|        4|    6625.0|      8000|         3|
+-----+---------+----------+----------+----------+



In [24]:
spark.stop()